In [ ]:
import findspark

In [ ]:
findspark.init()

In [ ]:
# Now import pyspark. And start SparkSession
import pyspark

In [ ]:
# The following three lines are required if you are running PySpark using Jupyter notebook with Python3 as kernel 
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark SQL Functions - Examples").getOrCreate()

In [ ]:
# Create DataFrame from data source - csv file
customerDF = spark.read.load("customers.txt", format="csv", sep="\t", inferSchema="true", header="true")

In [ ]:
customerDF.count()

In [ ]:
# Different types of operations on DataFrames

In [ ]:
customerDF.printSchema()

In [ ]:
customerDF.show()

In [ ]:
customerDF.select("customer_name","customer_city").show()

In [ ]:
customerDF.select(customerDF.customer_name,customerDF.customer_city).show(n=5)

In [ ]:
customerDF.select(customerDF['customer_name'], customerDF['customer_city']).show(5)

In [ ]:
customerDF.where(customerDF['customer_state'] == 'CA').show(5)

In [ ]:
df2=customerDF.where(customerDF['customer_state'] == 'CA')

In [ ]:
type(df2)

In [ ]:
df2.show(5)

In [ ]:
customerDF.groupBy("customer_state").count().show()

In [ ]:
# Aggregatons like sum, min, max, avg can be done as shown later below

In [ ]:
customerDF.groupBy("customer_state").count().orderBy("customer_state").show()

In [ ]:
from pyspark.sql.functions import *

In [ ]:
customerDF.groupBy("customer_state").count().orderBy(col("customer_state").desc()).show()

In [ ]:
customerDF.groupBy("customer_state").count().sort("customer_state").show()

In [ ]:
customerDF.groupBy("customer_state").count().sort(col("customer_state").desc()).show()

In [ ]:
productDF = spark.read.load("products.json", format="json")

In [ ]:
# Alternatively the following syntax is also allowed

# productDF = spark.read.json("products.json")

In [ ]:
# We can now run set of the DataFrame operations

In [ ]:
productDF.printSchema()

In [ ]:
productDF.select("product_name").show()

In [ ]:
productDF.select("product_name").show(truncate=False)

In [ ]:
productDF.select("product_name").show(5,False)

In [ ]:
productDF.select(productDF['product_name'], productDF['product_category'], productDF['product_price']).show()

In [ ]:
productDF.count()

In [ ]:
productDF.rdd.getNumPartitions()

In [ ]:
productDF.filter(productDF['product_price'] > 200.00).show(5)

In [ ]:
productDF.groupBy("product_category").count().show()

In [ ]:
productDF.groupBy("product_category").max("product_price").show(5)

In [ ]:
productDF.groupBy("product_category").avg("product_price").show(5)

# select productcategory, round(average(productprice),2) as avgprice group by productcategory

In [ ]:
productDF.groupBy("product_category").agg(count("product_price").alias("count_products")).show(5,False)

In [ ]:
productDF.groupBy("product_category").agg(round(avg("product_price"),2).alias("avg_price")).show(5,False)

In [ ]:
productDF.groupBy("product_category").\
    agg(\
        count("product_price").alias("count_products"),\
        avg("product_price").alias("avg_price"),\
        max("product_price").alias("max_price"),\
        min("product_price").alias("min_price")\
    ).\
    show(n=5,truncate=False)


In [ ]:
productDF.groupBy("product_category").\
    agg(\
        count("product_price").alias("count_products"),\
        round(avg("product_price"),2).alias("avg_price"),\
        max("product_price").alias("max_price"),\
        min("product_price").alias("min_price")\
    ).\
    show(5,truncate=False)


In [ ]:
# The following import statement is not required as we have used from pyspark.sql.functions import * above
# from pyspark.sql.functions import col, lit

In [ ]:
productDF.printSchema()

In [ ]:
# withColumn
## cast
prodDF2=productDF.withColumn('customer_id',col('customer_id').cast("Integer"))
prodDF2.printSchema()

In [ ]:
# withColumn
## add a new column with constant
prodDF3=prodDF2.withColumn('discount',lit(10.0))
prodDF3.printSchema()
prodDF3.show(5)

In [ ]:
# withColumn
## add a new column calculated with existing columns
prodDF4=prodDF3.withColumn('discounted_price',col('product_price')*(100-col('discount'))/100)
prodDF4.printSchema()
prodDF4.show(5)

In [ ]:
# withColumnRenamed
prodDF5=prodDF4.withColumnRenamed('customer_id','prodcust_id')
prodDF5.printSchema()
prodDF5.show(5)

In [ ]:
# Using alias to rename a column
prodDF5a = prodDF4.select(col('category_id'),col('customer_id').alias('prodcust_id'),\
                          col('product_category'),col('product_name'),col('product_price'),col('product_quantity'),\
                         col('salestxn_id'),col('discount'),col('discounted_price'))

prodDF5a.printSchema()
prodDF5a.show(5)

In [ ]:
customerDF.printSchema()
customerDF.show(2)

In [ ]:
prodDF5.printSchema()
prodDF5.show(2)

In [ ]:
# join
newDF=customerDF.join(prodDF5,customerDF.customer_id==prodDF5.prodcust_id,'outer')
newDF.printSchema()

In [ ]:
newDF.select('customer_id','customer_name','customer_state','prodcust_id','product_name','product_price','product_quantity').\
    where(newDF.customer_state=='IL').show()

In [ ]:
# df.na.fill on columns with a string type data
newDF2=newDF.na.fill(value='NA',subset=['product_name','product_category'])
# newDF2.printSchema()
newDF2.select('customer_id','customer_name','customer_state','product_category','prodcust_id','product_name','product_price','product_quantity').\
    where(newDF.customer_state=='IL').show()

In [ ]:
# df.na.fill on columns with an integer type data
newDF3=newDF2.na.fill(value=0,subset=['prodcust_id','category_id'])

In [ ]:
# newDF3.printSchema()

In [ ]:
newDF3.select('customer_id','customer_name','customer_state','customer_city','category_id','product_category','prodcust_id',\
              'product_name','product_price','product_quantity').where((newDF.customer_state=='IL') & (newDF.customer_city=='Chicago')).show()

In [ ]:
newDF.columns

In [ ]:
type(newDF.columns)

In [ ]:
len(newDF.columns)
# Gives you number of columns in the dataframe

In [ ]:
newDF.count()
# Gives you number of rows in the dataframe as you know